In [1]:
%pip install semantic-link --quiet 
import synapse.ml.core
import pandas as pd
from synapse.ml.services import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, flatten, udf, lower, trim
from pyspark.sql.types import StringType
import sempy
import sempy.fabric as fabric
import base64
import json
import copy

StatementMeta(, 4e1e0804-5f31-4732-b488-215d2f395c36, 8, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



# Parameters & initialize

In [2]:
# Parameters
workspaceId = fabric.get_workspace_id()
reportId = "054e3ecf-56d4-4da5-9610-af4148fc839a"

languages = ["fr", "it"]
#Hindi-hi; Hungarian-hu;Maori-mi;Thai-th

newReportName = "[trans] - Sales"

# Client initialization
client = fabric.FabricRestClient()

StatementMeta(, 4e1e0804-5f31-4732-b488-215d2f395c36, 10, Finished, Available)

# Cleanup for the demo

In [4]:
workspaceId = fabric.get_workspace_id()
response = client.get(f"/v1/workspaces/{workspaceId}/items")

listItems = response.json()['value']

itemsToDelete = [x for x in listItems if x['type'] == 'Report' and x['displayName'].startswith(newReportName)]

for item in itemsToDelete:
    itemId = item['id']
    
    print(f"Deleting report '{itemId}'")

    client.delete(f"/v1/workspaces/{workspaceId}/items/{itemId}")

StatementMeta(, 93e4147c-032f-4a44-be3e-805c638a13d8, 12, Finished, Available)

Deleting report 'b15a7df5-a09c-4044-9650-429aff8a15e9'
Deleting report 'e761e420-fc9a-4b4e-b75e-3d896a72a1fc'
Deleting report 'a7829825-9748-427b-837a-8e637b1dcbc2'
Deleting report 'cfcb6006-eab0-422f-b2fd-1af1a9c2133a'
Deleting report 'f546a659-d021-485c-87a3-6438cf6a4f5b'
Deleting report '377a8233-19cb-4756-ae03-6f136795dd82'
Deleting report '71adf4af-2d24-4813-b6e5-fea927f695fe'


# Get report definition parts

In [3]:
response = client.post(f"/v1/workspaces/{workspaceId}/items/{reportId}/getDefinition")

jsonParts = response.json()['definition']['parts']

df_parts = pd.json_normalize(jsonParts)
pd.set_option('display.max_colwidth', 100) 
df_parts

StatementMeta(, 4e1e0804-5f31-4732-b488-215d2f395c36, 11, Finished, Available)

,path,payload,payloadType
0,definition.pbir,ewogICJ2ZXJzaW9uIjogIjQuMCIsCiAgImRhdGFzZXRSZWZlcmVuY2UiOiB7CiAgICAiYnlQYXRoIjogbnVsbCwKICAgICJi...,InlineBase64
1,StaticResources/SharedResources/BaseThemes/CY23SU04.json,ewogICJuYW1lIjogIkNZMjNTVTA0IiwKICAiZGF0YUNvbG9ycyI6IFsKICAgICIjMTE4REZGIiwKICAgICIjMTIyMzlFIiwK...,InlineBase64
2,StaticResources/RegisteredResources/Light4437032645752863.json,ewogICIkc2NoZW1hIjogImh0dHBzOi8vcmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbS9taWNyb3NvZnQvcG93ZXJiaS1kZXNr...,InlineBase64
3,StaticResources/RegisteredResources/_7abfc6c7-1a23-4b5f-bd8b-8dc472366284171093267.jpg,77+9UE5HDQoaCgAAAA1JSERSAAACAAAAAgAIBgAAAO+/vXjvv73vv70AAAAJcEhZcwAACxMAAAsTAQDvv73vv70YAAAAAXNS...,InlineBase64
4,definition/version.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
5,definition/report.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
6,definition/pages/pages.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
7,definition/pages/ReportSection89a9619c7025093ade1c/page.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
8,definition/pages/ReportSection89a9619c7025093ade1c/visuals/0f9bcdc145fc9e6fdc33/visual.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64
9,definition/pages/ReportSection89a9619c7025093ade1c/visuals/223d9300700274da67c8/visual.json,ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRl...,InlineBase64


# Enumerate all the report visuals **with** titles

In [ ]:
# Get the report json part and parse it

reportPart = [part for part in jsonParts if part["path"] == 'report.json'][0]

payload = reportPart['payload']

reportFile = base64.b64decode(payload).decode('utf-8')

reportJson = json.loads(reportFile)

# Initialized data frame to load all visuals + current title

visualsDF = pd.DataFrame({'visualID': [], 'title': []})

# Build visual + title dictionary (if exists)

for section in reportJson['sections']:

    for visual in section['visualContainers']:
        visualConfig = visual['config']
        visualConfigJson = json.loads(visualConfig)        
        visualID = visualConfigJson['name']
        try:
            title = visualConfigJson["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]["Value"]
        except:
            title = ""

        if title != "":
            title = title.strip("'")
            visualsDF = pd.concat([visualsDF, pd.DataFrame({'visualID': visualID, 'title': title}, index=[0])], ignore_index=True)

visualsDF

StatementMeta(, e90b4545-4c2f-41f9-a4df-aea4c22098db, 12, Finished, Available)

,visualID,title
0,32b08d7f32cb5f9816cc,Sales Amount by Brand and Gender
1,484fbdd73143c5bf71fa,"Sales Amount, Sales Amount (LY), Margin and Sales Amount (12M average) by Date (Year-Month)"
2,4b18758374093c2e23fb,Sales Amount vs # Customers
3,5acb1caf298449a8acb4,Sales Amount and Sales Amount (LY) by Category
4,6f9803040a3195450ea1,Value by Product
5,e50725a866c432900b35,Value by Date
6,e859ee4d7c5b10e836d6,"Value, Value (ly) and Value Daily Max by Month Start Date"
7,f0143b84500069035233,Value (ytd) by Month Start Date
8,a64038428c095bd71739,Sales Amount by Country and Gender


# Call Azure AI translator 

In [ ]:
# Initialize Azure AI translator

translate = (Translate()
    .setTextCol("title")
    .setToLanguage(languages)
    .setOutputCol("translation")
    .setConcurrency(5))

# Call Azure AI translator to translate all the visual titles into multiple languages

visualsDF_spark = sqlContext.createDataFrame(visualsDF)

translatedDF = translate.transform(visualsDF_spark)\
        .withColumn("translation_result", flatten(col("translation.translations")))\
        .withColumn("translation", col("translation_result.text")[0])\
        .cache()

display(translatedDF)

translatedDF = translatedDF.toPandas()

StatementMeta(, e90b4545-4c2f-41f9-a4df-aea4c22098db, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9e1b5992-ee71-4e30-8aa1-e78ec8a96767)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:117: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.


# For each language: create a report with the translated visuals

In [ ]:
for lang in languages:

    reportJsonLang = copy.deepcopy(reportJson)

    translatedReportName = f"{newReportName} - {lang}"

    print(f"Creating report '{translatedReportName}'")

    for section in reportJsonLang['sections']:

        for visual in section['visualContainers']:

            visualConfig = visual['config']
            visualConfigJson = json.loads(visualConfig)  
            visualID = visualConfigJson['name']
            
            # Find translation for the visual

            translatedTexts = translatedDF.loc[translatedDF['visualID'] == visualID, 'translation_result']
            
            if not translatedTexts.empty:

                # Get translation of the visual for the language

                translatedText = [t for t in translatedTexts.values[0] if t.to == lang][0]
                
                if translatedText is not None:
                    try:
                        
                        # Get visual title json property

                        titleProperty = visualConfigJson["singleVisual"]["vcObjects"]["title"][0]["properties"]["text"]["expr"]["Literal"]

                        translatedTextStr = translatedText.text
                    
                        titleProperty["Value"] = f"'{translatedTextStr}'"            
            
                    except KeyError:
                        pass
            
            # Replace the visual 'config' json

            visual['config'] = json.dumps(visualConfigJson)

    # Create a new report by replacing the report.json part

    # Get all parts except report.json (we will add a new one)

    jsonPartsLang = [part for part in jsonParts if part["path"] != 'report.json']

    reportPartLang = copy.deepcopy(reportPart)

    reportPartLang['payload'] = base64.b64encode(json.dumps(reportJsonLang).encode('utf-8')).decode('utf-8')

    jsonPartsLang.append(reportPartLang)

    payload = {}
    payload['displayName'] = translatedReportName
    payload['type'] = "Report"
    payload['definition'] =  {
        'parts' : jsonPartsLang
    }

    response = client.post(f"/v1/workspaces/{workspaceId}/items", json= payload)

StatementMeta(, e90b4545-4c2f-41f9-a4df-aea4c22098db, 14, Finished, Available)

Creating report '[trans] - Sales - fr'
Creating report '[trans] - Sales - it'
Creating report '[trans] - Sales - pt'
Creating report '[trans] - Sales - ja'
Creating report '[trans] - Sales - th'
Creating report '[trans] - Sales - de'
Creating report '[trans] - Sales - hu'
